# Install the necessary modules

In [0]:
!pip install langchain-mcp-adapters langchain databricks-langchain --quiet


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-api-core 2.18.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0.dev0,>=3.19.5, but you have protobuf 5.29.5 which is incompatible.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


# Restart the kernel

In [0]:
dbutils.library.restartPython()

# Perform necessary imports

In [0]:
from langchain_mcp_adapters.client import MultiServerMCPClient  
from langchain.agents import create_agent

# [Define the client ](https://docs.langchain.com/oss/python/langchain/mcp#quickstart)

In [0]:
client = MultiServerMCPClient(  
    {
        "slack_linkedin_post": {
            "transport": "http",
            "url": "{}",
        },
        "email_post": {
            "transport": "http",
            "url": "{}",
        }
    }
)

# Get the available tools to access

In [0]:
tools = await client.get_tools()
tools

[StructuredTool(name='add_tools', description='Add new actions to your MCP provider', args_schema={'type': 'object', 'properties': {}, 'required': []}, metadata={'title': 'Add tools', 'readOnlyHint': None, 'destructiveHint': None, 'idempotentHint': None, 'openWorldHint': None}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x7f28a047d080>),
 StructuredTool(name='edit_tools', description='Edit your existing MCP provider actions', args_schema={'type': 'object', 'properties': {}, 'required': []}, metadata={'title': 'Edit tools', 'readOnlyHint': None, 'destructiveHint': None, 'idempotentHint': None, 'openWorldHint': None}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x7f28a04ca8e0>),
 StructuredTool(name='slack_send_channel_message', description='Post a message or thread reply to a channel. You can also schedule it to send later.', args_schema={'ty

# Define the LLM

In [0]:
from databricks_langchain import ChatDatabricks
llm = ChatDatabricks(endpoint="databricks-gemma-3-12b")

# Create the Agent

In [0]:
agent = create_agent(llm, tools)

# Try slack tool 

In [0]:
response = await agent.ainvoke({"messages": "Put a message in test_zapier_mcp slack channel explaining brief about 2011 Cricket World Cup."})

[Trace(trace_id=tr-723c906fc54d6d273d619807dcc7d345), Trace(trace_id=tr-ebfc30fabda2238dc51dd96bce9ee79d), Trace(trace_id=tr-fd47e18040e444d9389cc6ad870d677a)]

# Try LinkedIn tool

In [0]:
response = await agent.ainvoke({"messages": "Write a LinkedIn Post mentioning that you are feeling good today."})

[Trace(trace_id=tr-5d3cb33e271a4cd54d716879a6eceb13), Trace(trace_id=tr-6dd324c7e95227ff164b3dbf1196f2e9), Trace(trace_id=tr-3a69bcffaa8fa7adadac8ab1afd5c82a)]

In [0]:
response

{'messages': [HumanMessage(content='Write a LinkedIn Post mentioning that you are feeling good today.', additional_kwargs={}, response_metadata={}, id='8c12fb94-9738-44e4-b2d1-f9b54fb1c6d0'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_0c4c62fe-becf-4c0a-9545-95ee60c43ea7', 'function': {'arguments': '{"instructions": "Create a LinkedIn post about feeling good today.", "output_hint": "just the post text"}', 'name': 'linkedin_create_share_update'}, 'type': 'function'}]}, response_metadata={'usage': {'prompt_tokens': 2341, 'completion_tokens': 30, 'total_tokens': 2371}, 'prompt_tokens': 2341, 'completion_tokens': 30, 'total_tokens': 2371, 'model': 'gemma-3-12b-it-060225', 'model_name': 'gemma-3-12b-it-060225', 'finish_reason': 'tool_calls'}, id='lc_run--019b474b-7ce0-7121-b4df-ab8a367e555d-0', tool_calls=[{'name': 'linkedin_create_share_update', 'args': {'instructions': 'Create a LinkedIn post about feeling good today.', 'output_hint': 'just the post text'}, 'id

# Try Email sending tool

In [0]:
response = await agent.ainvoke({"messages": "Send an email to address ghinku12345@gmail.com with some information about great things what Modi Govt has done in recent years, mention 5 key points."})
response

{'messages': [HumanMessage(content='Send an email to address ghinku12345@gmail.com with some information about great things what Modi Govt has done in recent years, mention 5 key points.', additional_kwargs={}, response_metadata={}, id='aaa96e8c-c36b-46ce-9a5b-e867bee7fbe7'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_213869b4-7db7-4d45-87e1-6325c13cf059', 'function': {'arguments': '{"instructions": "Send an email to address ghinku12345@gmail.com with some information about great things what Modi Govt has done in recent years, mention 5 key points.", "to": ["ghinku12345@gmail.com"], "subject": "Modi Government Achievements", "body": "Dear Recipient,\\n\\nHere are 5 key achievements of the Modi Government in recent years:\\n\\n1. Infrastructure Development: Significant progress in building roads, highways, and railways.\\n2. Digital India Initiative: Expanded internet access and promoted digital literacy.\\n3. Financial Inclusion: Initiatives like Jan Dhan Yo

[Trace(trace_id=tr-1e10eb69000c99c916ee03ddaa030adc), Trace(trace_id=tr-7f0e510f673a1f32f4f0c395697d4404), Trace(trace_id=tr-fc2571edb4fbb8e7a1deae01cab8710f)]